### Necessary imports

In [ ]:
# Install all dependencies
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7


### Dependencies

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

### Load quantized Mistal 7B

In [3]:
#################################################################
# Tokenizer
################

model_name='mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
# if compute_dtype == torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("=" * 80)
#         print("Your GPU supports bfloat16: accelerate training with bf16=True")
#         print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


### Count number of trainable parameters

In [4]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


### Build Mistral text generation pipeline

In [5]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [6]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [7]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

### Load and chunk documents. Load chunked documents into FAISS index

In [ ]:
!playwright install
!playwright install-deps

In [ ]:
import pandas as pd
# import io
# from google.colab import files
# uploaded = files.upload()
df = pd.read_csv('/content/remarks&subject_content.csv')

In [ ]:
# Printing the Dataset
print(df)


                                         remarks_text  \
0   \r\nमहोदय/महोदया, \r\nहम इस संदर्भ में आपको अव...   
1    Customer concern is regarding slow browsing c...   
2    \tPlease provide the name of ISP of internet ...   
3    \r\nThe Grievance has been sent to the State ...   
4           The petitioner has been suitably advised.   
5           The petitioner has been suitably advised.   
6           The petitioner has been suitably advised.   
7                    As per report of the Department.   
8                    As per report of the Department.   
9                    As per report of the Department.   
10                   As per report of the Department.   
11                   As per report of the Department.   
12                         Contact to circle office     
13     Debit card has been issued and satisfaction...   
14     The Role of General Administration Departme...   
15                  Action taken by the concerned AD.   
16    As per psp system remarks

In [ ]:
df.head(24)

,remarks_text,subject_content_text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,"\r\nमहोदय/महोदया, \r\nहम इस संदर्भ में आपको अव...",Financial Services (Banking Division) >> Fraud...,NaN,NaN,NaN
1,Customer concern is regarding slow browsing c...,Telecommunications >> Broadband Related >> Oth...,NaN,NaN,NaN
2,\tPlease provide the name of ISP of internet ...,My call drop and internet is not working for t...,NaN,NaN,NaN
3,\r\nThe Grievance has been sent to the State ...,Food and Public Distribution >> Public Distrib...,NaN,NaN,NaN
4,The petitioner has been suitably advised.,December month pension not come.,NaN,NaN,NaN
5,The petitioner has been suitably advised.,Though I have diposited my life certificate in...,NaN,NaN,NaN
6,The petitioner has been suitably advised.,वित्तीय सेवा विभाग (बैंकिंग प्रभाग) >> Miscell...,NaN,NaN,NaN
7,As per report of the Department.,Agriculture and Farmers Welfare >> Crops relat...,NaN,NaN,NaN
8,As per report of the Department.,Agriculture and Farmers Welfare >> PMKISAN rel...,NaN,NaN,NaN
9,As per report of the Department.,सर् नमस्कार मै PM KISAN SAMMAN NIDHI का लाभार्...,NaN,NaN,NaN


In [9]:
# !pip install PyPDF2
# import PyPDF2
# import os
# file_paths = [
#   '/content/Grievances.Dataset.pdf'
# ]

# text = ""
# for i in file_paths:
#   pdfFile = open(i,'rb')
#   Reader = PyPDF2.PdfReader(pdfFile)

#   for page in range(len(Reader.pages)):
#     pageObj = Reader.pages[page]
#     text += pageObj.extract_text()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [10]:
class Document:
    def __init__(self, content):
        self.page_content = content
        self.metadata= {"source":"gmail"}

In [ ]:
# Passing URLs and PDF Document for Retrieval-Augmented Generation (RAG) to assist the LLM

# !pip install PyPDF2
import PyPDF2
import nest_asyncio
import imaplib
import email
nest_asyncio.apply()

attachment=""
# Articles to index
articles = [
            " https://www.pgportal.gov.in/",
            "https://www.pgportal.gov.in/Home/Faq",
            "https://www.pgportal.gov.in/Home/AboutUs",
            "https://www.pgportal.gov.in/Home/ContactUs",


]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()
print(docs)
    # Converts HTML to plain text
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
print(docs_transformed)
  # Chunk text

text_splitter = CharacterTextSplitter(chunk_size=100,
                                        chunk_overlap=0)

# Read Grievances PDF Document for RAG so that LLM can understand the grievances faced by people.
file_paths = [
  '/content/Grievances.Dataset.pdf'
]

text = ""
for i in file_paths:
  pdfFile = open(i,'rb')
  Reader = PyPDF2.PdfReader(pdfFile)

  for page in range(len(Reader.pages)):
    pageObj = Reader.pages[page]
    text += pageObj.extract_text()

import langchain_core
# Prepare Document object (modify based on actual requirements)
document = langchain_core.documents.base.Document(page_content=text)
# document.text = text  # Assuming there's a "text" attribute in the Document class
docs_transformed.append(document)
print(docs_transformed)

chunked_documents=text_splitter.split_documents(docs_transformed)

#   # Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents,
                            HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()


In [13]:
print(chunked_documents)

[Document(page_content='* भारत सरकार  Government of India\n  * कार्मिक, लोक शिकायत और पेंशन मंत्रालय Ministry of Personnel, Public Grievances & Pensions', metadata={'source': ' https://www.pgportal.gov.in/'}), Document(page_content='* Home\n  * Contact Us\n  * About Us\n  * FAQs/Help\n  * Site Map\n  * 2024 Holiday List\n\n# CPGRAMS', metadata={'source': ' https://www.pgportal.gov.in/'}), Document(page_content='Centralized Public Grievance Redress And Monitoring System', metadata={'source': ' https://www.pgportal.gov.in/'}), Document(page_content='* View Status  Grievance Status Appeal Status\n  * Nodal PG Officers  Central Government State Government\n  * Redress Process  Redress Process Flow\n  * Grievance  Lodge Public Grievance Lodge Pension Grievance View Status Reminder Clarification Rate Grievance\n  * __Nodal Authority for Appeal(current)\n  * Mobile App', metadata={'source': ' https://www.pgportal.gov.in/'}), Document(page_content='Language :\n\nEnglish', metadata={'source': '

In [14]:
# This document contains the extracted text
print(docs_transformed)

[Document(page_content="  * भारत सरकार  Government of India\n  * कार्मिक, लोक शिकायत और पेंशन मंत्रालय Ministry of Personnel, Public Grievances & Pensions\n\n  * Home\n  * Contact Us\n  * About Us\n  * FAQs/Help\n  * Site Map\n  * 2024 Holiday List\n\n# CPGRAMS\n\nCentralized Public Grievance Redress And Monitoring System\n\n\n\n  * View Status  Grievance Status Appeal Status\n  * Nodal PG Officers  Central Government State Government\n  * Redress Process  Redress Process Flow\n  * Grievance  Lodge Public Grievance Lodge Pension Grievance View Status Reminder Clarification Rate Grievance\n  * __Nodal Authority for Appeal(current)\n  * Mobile App \n\nLanguage :\n\nEnglish\n\nEnglish हिंदी (Hindi) ગુજરાતી (Gujarati) मराठी (Marathi) বাংলা (Bangala)\nతెలుగు (Telugu) অসমীয়া (Assamese) ଓଡିଆ (Odia) தமிழ் (Tamil) മലയാളം (Malayalam)\n(Urdu) اردو Sindhi बोडो (Bodo) कोंकणी (Konkani) नेपाली (Nepali) Manipuri\nਪੰਜਾਬੀ (Punjabi) ಕನ್ನಡ (Kannada) डोगरी (Dogri) मैथिली (Maithili) کشمیر\n(Kashmiri) संस्कृ

### Create PromptTemplate and LLMChain

In [15]:
prompt_template = """
### [INST] Instruction: You Are a Centralized Public Grievance Redress And Monitoring System Knowledge ChatBot, Reply to each query accurately by giving detailed answers relevant to the context provided. Here is the context to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

### Build RAG Chain

In [16]:

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)
result = rag_chain.invoke("List some grievances faced?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [17]:
result['context']

[Document(page_content='10\\. What are the types of grievances which are not taken up for redress by\nthe Department?', metadata={'source': 'https://www.pgportal.gov.in/Home/Faq'}),
 Document(page_content='6\\. What happens to the grievances? How are the grievances dealt with in\nCentral Ministries/Departments?', metadata={'source': 'https://www.pgportal.gov.in/Home/Faq'}),
 Document(page_content='15\\. What can a citizen do if he is not satisfied with the redressal of his\ngrievance?', metadata={'source': 'https://www.pgportal.gov.in/Home/Faq'}),
 Document(page_content='2\\. Where can the grievances be sent?\n\nThe grievances can be sent to :', metadata={'source': 'https://www.pgportal.gov.in/Home/Faq'})]

In [18]:
print(result['text'])

1. Grievances related to corruption and maladministration
 2. Grievances related to non-delivery of public services
 3. Grievances related to violation of human rights
 4. Grievances related to environmental degradation
 5. Grievances related to social justice and equality
 6. Grievances related to health and education
 7. Grievances related to employment and labor issues
 8. Grievances related to taxation and financial matters
 9. Grievances related to security and defense issues
 10. Grievances related to foreign policy and diplomacy issues


In [ ]:
!pip install gradio

In [ ]:
from transformers import pipeline
p = pipeline("automatic-speech-recognition")

In [21]:
import gradio as gr

In [22]:
# first function ~ used to transcribe (speech to text)
def transcribe(audio):
    transcribed_text = p(audio)["text"]
    result = rag_chain.invoke(transcribed_text)
    return result['text']

In [23]:
# second function
def chat(chat_history, user_input):

  result = rag_chain.invoke(user_input)

  response = ""
  for letter in ''.join(result['text']): #[bot_response[i:i+1] for i in range(0, len(bot_response), 1)]:
      response += letter + ""
      yield chat_history + [(user_input, response)]

In [ ]:
# Creating an interactive User Interface for a Chat Assistant that can take Audio & Text as input

import gradio as gr
with gr.Blocks() as demo:
    with gr.Tab("Text"):
          chatbot = gr.Chatbot()
          message = gr.Textbox ("What is this document about?")
          message.submit(chat,[chatbot, message], chatbot)
    with gr.Tab ("Audio"):
      gr.Interface(
    fn=transcribe,
    inputs=[
        gr.Audio(type="filepath")

    ],
    outputs=[
        "textbox"

    ], live=True)
    demo.queue().launch(debug = True)